<a href="https://colab.research.google.com/github/AnaKarenDRIV/FinanzasUniversitarias/blob/main/RedesConvusionales/%20Clasificaci%C3%B3n_de_Ropa(comentado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Nota: ejecutar en un entorno con TensorFlow instalado (GPU opcional).

import numpy as np  # manejo de arreglos y operaciones numéricas
import tensorflow as tf  # framework de deep learning
from tensorflow.keras.models import Sequential  # modelo secuencial (capas apiladas)
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.optimizers import Adam  # optimizador Adam
from tensorflow.keras.datasets import fashion_mnist  # dataset Fashion MNIST
from tensorflow.keras.losses import SparseCategoricalCrossentropy  # pérdida para etiquetas enteras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # evaluación: matriz de confusión
import seaborn as sns  # visualización estadística (opcional)
import matplotlib.pyplot as plt  # visualizaciones con matplotlib

# -------------------------
# 1) Información y carga del dataset
# -------------------------
# El dataset Fashion MNIST ya viene dividido en entrenamiento y prueba por defecto.
# Cada imagen es de 28x28 píxeles en escala de grises.

# Referencia: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data

# Definimos nombres de clases (el índice corresponde a la etiqueta numérica)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Cargar los datos (devuelve tuplas para train y test)
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# -------------------------
# 2) Exploración rápida / visualización
# -------------------------
# Mostrar una imagen de ejemplo y su etiqueta para verificar que los datos se cargaron bien.
index = 10  # índice de ejemplo a visualizar
image = train_images[index]
label = train_labels[index]

# Imprimir la matriz numérica (28x28) que representa la imagen
print(image)

# Mostrar la imagen con matplotlib (mapa de grises)
plt.figure(figsize=(3, 3))
plt.imshow(image, cmap=plt.cm.gray)
plt.title(f'Etiqueta número: {label} -> {class_names[label]}')
plt.axis('off')
plt.show()

# -------------------------
# 3) Preprocesamiento
# -------------------------
# Revisar formas de los arreglos
print('Forma train_images:', train_images.shape)  # (60000, 28, 28)
print('Forma test_images: ', test_images.shape)   # (10000, 28, 28)

# Normalizar los valores de píxel a [0, 1] para estabilizar el entrenamiento
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Keras Conv2D espera entradas con forma (alto, ancho, canales). Actualmente las imágenes son (28,28).
# Agregamos la dimensión de canales = 1 para indicar que son en escala de grises.
train_images = np.expand_dims(train_images, -1)  # ahora (60000, 28, 28, 1)
test_images = np.expand_dims(test_images, -1)    # ahora (10000, 28, 28, 1)

# -------------------------
# 4) Definición del modelo (CNN)
# -------------------------
# Construimos un modelo secuencial con varias capas convolucionales y capas densas al final.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # 1a conv: 32 filtros 3x3
    MaxPooling2D((2, 2)),  # primer max-pool reduce la resolución espacial a la mitad
    Conv2D(64, (3, 3), activation='relu'),  # 2a conv: 64 filtros
    MaxPooling2D((2, 2)),  # segundo max-pool
    Conv2D(64, (3, 3), activation='relu'),  # 3a conv: 64 filtros
    Flatten(),  # aplana los mapas de características a un vector
    Dense(64, activation='relu'),  # capa totalmente conectada intermedia
    Dense(10)  # capa de salida: 10 unidades (logits). No aplicamos softmax aquí.
])

# Resumen del modelo para revisar capas y parámetros
model.summary()

# -------------------------
# 5) Compilación
# -------------------------
# Definimos optimizador Adam con tasa de aprendizaje especificada
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

# Compilar el modelo: indicamos optimizador, función de pérdida (from_logits=True porque no usamos softmax)
model.compile(optimizer=adam_optimizer,
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# -------------------------
# 6) Entrenamiento
# -------------------------
# Ajustar (entrenar) el modelo con los datos de entrenamiento y validar en el conjunto de prueba
# Nota: se puede ajustar batch_size, callbacks (EarlyStopping, ModelCheckpoint), etc.
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# -------------------------
# 7) Evaluación final
# -------------------------
# Evaluar el modelo en el conjunto de prueba para obtener pérdida y precisión
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Pérdida en test: {test_loss:.4f} - Precisión en test: {test_acc:.4f}')

# -------------------------
# 8) Predicciones y matriz de confusión
# -------------------------
# Construimos un modelo que incluye softmax para convertir logits en probabilidades
probability_model = Sequential([model, Softmax()])

# Obtener predicciones (probabilidades) para todas las imágenes de test
predictions = probability_model.predict(test_images)
# Convertir probabilidades a etiquetas predichas (índice de máxima probabilidad)
predicted_labels = np.argmax(predictions, axis=1)

# Calcular la matriz de confusión usando sklearn
cm = confusion_matrix(test_labels, predicted_labels)

# Mostrar la matriz de confusión (gráfico grande para mejor lectura)
fig, ax = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(ax=ax, xticks_rotation='vertical')
plt.title('Matriz de confusión - Fashion MNIST')
plt.show()

# -------------------------
# 9) BONUS: función para graficar imágenes con predicción
# -------------------------
def plot_image(i, predictions_array, true_label, img):
    """Dibuja la imagen i junto con la etiqueta predicha y la verdadera.
    predictions_array: vector de probabilidades (para una sola imagen).
    true_label: arreglo de etiquetas verdaderas.
    img: arreglo de imágenes (sin normalizar o ya normalizado está bien).
    """
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    # img puede venir con forma (28,28,1) o (28,28)
    plt.imshow(img.reshape((28, 28)), cmap=plt.cm.binary)

    # índice de la clase predicha
    predicted_label = np.argmax(predictions_array)
    # color azul si acierta, rojo si falla
    color = 'blue' if predicted_label == true_label else 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(
        class_names[predicted_label],
        100 * np.max(predictions_array),
        class_names[true_label]
    ), color=color)

# Graficar varias predicciones de ejemplo
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], test_labels, test_images)

plt.tight_layout()
plt.show()
